In [1]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [4]:
#TRAIN_DIR = '/content/drive/MyDrive/Enhanced training data/TRAIN' #train enhanced directory path
#TEST_DIR = '/content/drive/MyDrive/Enhanced training data/TEST'   #test enhanced directory path

TRAIN_DIR = '/content/drive/MyDrive/SFEW_1.0/TRAIN'  #train normal dir path
TEST_DIR = '/content/drive/MyDrive/SFEW_1.0/TEST'   #test normal directory path

In [5]:
#image resolutions
HEIGHT = 48
WIDTH = 48

In [6]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "created")
    return image_paths,labels

In [7]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

Fear created
Disgust created
Angry created
Surprise created
Happy created
Sad created
Neutral created


In [8]:
print(train)

                                                 image    label
0    /content/drive/MyDrive/SFEW_1.0/TRAIN/Fear/10.jpg     Fear
1     /content/drive/MyDrive/SFEW_1.0/TRAIN/Fear/1.jpg     Fear
2    /content/drive/MyDrive/SFEW_1.0/TRAIN/Fear/27.jpg     Fear
3    /content/drive/MyDrive/SFEW_1.0/TRAIN/Fear/20.jpg     Fear
4    /content/drive/MyDrive/SFEW_1.0/TRAIN/Fear/25.jpg     Fear
..                                                 ...      ...
341  /content/drive/MyDrive/SFEW_1.0/TRAIN/Neutral/...  Neutral
342  /content/drive/MyDrive/SFEW_1.0/TRAIN/Neutral/...  Neutral
343  /content/drive/MyDrive/SFEW_1.0/TRAIN/Neutral/...  Neutral
344  /content/drive/MyDrive/SFEW_1.0/TRAIN/Neutral/...  Neutral
345  /content/drive/MyDrive/SFEW_1.0/TRAIN/Neutral/...  Neutral

[346 rows x 2 columns]


In [9]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

Angry created
Neutral created
Disgust created
Sad created
Happy created
Surprise created
Fear created


In [10]:
print(test)

                                                 image  label
0    /content/drive/MyDrive/SFEW_1.0/TEST/Angry/12.jpg  Angry
1    /content/drive/MyDrive/SFEW_1.0/TEST/Angry/14.jpg  Angry
2     /content/drive/MyDrive/SFEW_1.0/TEST/Angry/1.jpg  Angry
3    /content/drive/MyDrive/SFEW_1.0/TEST/Angry/20.jpg  Angry
4    /content/drive/MyDrive/SFEW_1.0/TEST/Angry/17.jpg  Angry
..                                                 ...    ...
349   /content/drive/MyDrive/SFEW_1.0/TEST/Fear/48.jpg   Fear
350   /content/drive/MyDrive/SFEW_1.0/TEST/Fear/46.jpg   Fear
351   /content/drive/MyDrive/SFEW_1.0/TEST/Fear/39.jpg   Fear
352    /content/drive/MyDrive/SFEW_1.0/TEST/Fear/8.jpg   Fear
353   /content/drive/MyDrive/SFEW_1.0/TEST/Fear/44.jpg   Fear

[354 rows x 2 columns]


In [11]:
from tqdm.notebook import tqdm

In [ ]:
from PIL import Image

In [ ]:

# def resize(image):
#   img = load_img(image)
#   img_resized = img.resize((WIDTH, HEIGHT))
#   return img_resized

In [12]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = "grayscale" )
        img = img.resize((WIDTH , HEIGHT))  #resizing image
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),HEIGHT,WIDTH,1)
    return features


In [13]:
train_features = extract_features(train['image'])

  0%|          | 0/346 [00:00<?, ?it/s]

In [14]:
test_features = extract_features(test['image'])

  0%|          | 0/354 [00:00<?, ?it/s]

In [15]:
x_train = train_features/255.0
x_test = test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [20]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(HEIGHT,WIDTH,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [21]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [31]:
model.fit(x= x_train,y = y_train, batch_size = 20, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
18/18 [==============================] - 0s 26ms/step - loss: 0.0556 - accuracy: 0.9884 - val_loss: 6.8062 - val_accuracy: 0.1921
Epoch 2/100
18/18 [==============================] - 0s 19ms/step - loss: 0.0379 - accuracy: 0.9855 - val_loss: 6.6704 - val_accuracy: 0.2006
Epoch 3/100
18/18 [==============================] - 0s 19ms/step - loss: 0.0998 - accuracy: 0.9855 - val_loss: 5.9159 - val_accuracy: 0.1864
Epoch 4/100
18/18 [==============================] - 0s 23ms/step - loss: 0.0260 - accuracy: 0.9942 - val_loss: 5.8611 - val_accuracy: 0.1977
Epoch 5/100
18/18 [==============================] - 0s 19ms/step - loss: 0.0488 - accuracy: 0.9855 - val_loss: 6.2623 - val_accuracy: 0.1921
Epoch 6/100
18/18 [==============================] - 0s 19ms/step - loss: 0.0351 - accuracy: 0.9913 - val_loss: 6.6152 - val_accuracy: 0.2175
Epoch 7/100
18/18 [==============================] - 0s 23ms/step - loss: 0.0949 - accuracy: 0.9798 - val_loss: 6.4519 - val_accuracy: 0.2147
Epoch 

**Save the model**

In [34]:
model_json = model.to_json()
with open("emotiondetectornormal48*48.json",'w') as json_file:
    json_file.write(model_json)

model.save("/content/drive/MyDrive/emotiondetectornormal48*48.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [32]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)


12/12 [==============================] - 0s 7ms/step - loss: 6.4568 - accuracy: 0.2062
Test accuracy: 0.20621469616889954
